In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

In [130]:
with open('../templateMatching/newVersionJsonOutput.txt') as data_file:
    testOutput = json.load(data_file)
with open('../../Desktop/onePageInfo.json') as data_file:
    groundTruth = json.load(data_file)
resultsDF = pd.DataFrame(testOutput)
gtDF = pd.DataFrame(groundTruth)
print resultsDF.shape
gtDF.head(10)
resultsDF.head(20)
#print resultsDF.head(10)

(2043, 19)


,30-Day Supply,90-Day Supply,Already Sent a Refill,Have Patient Contact My Office,No Longer a Patient,Number of Additional Refills,Other Reason for Denial,Patient Unknown to this Office,Refill Too Soon,digitConfidences,digitPictureNumber,foundALine,imageName,maxDigitConf,numberOfPictures,predictedDigit,templateLocation,templateMatchValue,templateRatio
0,0,0,4531,0,175,0,15903,0,18054,"[[-59.8352622986, 250.931015015, -281.70867919...",5,False,../myNewFaxes/5935dfdb91b5c97741037eea,1267.106201,6,7,"[0, 0]",1.000000,"[0.85, 0.85]"
1,2048,21949,0,0,0,23229,0,0,0,"[[648.535461426, 936.109313965, -343.333496094...",3,True,../myNewFaxes/5931d010acdd3f4b517caa7e,4097.559570,6,3,"[366, 44]",0.596617,"[0.994827586207, 0.984482758621]"
2,0,21691,0,0,0,10714,0,0,0,"[[228.354995728, 497.902252197, -466.080993652...",2,False,../myNewFaxes/594ae05891b5c95896d61a41,2693.408936,6,1,"[343, 35]",0.440446,"[0.984482758621, 0.984482758621]"
3,0,21432,0,0,0,0,0,0,0,"[[562.207702637, 304.85723877, -35.170337677, ...",5,False,../myNewFaxes/5940400b91b5c94d9a25185b,1215.128906,6,2,"[369, 30]",0.568384,"[0.984482758621, 0.994827586207]"
4,0,7771,0,0,0,88,319,0,0,"[[-172.149337769, 306.297058105, -252.70153808...",5,False,../myNewFaxes/5914a87d91b5c946cd1d8c11,1247.911865,6,7,"[407, 94]",0.460527,"[0.953448275862, 0.93275862069]"
5,22897,6511,0,256,0,37008,128,0,0,"[[-1043.94958496, 195.873092651, 1181.68444824...",2,True,../myNewFaxes/593efd033839be1fa4d94d53,1934.181152,6,2,"[358, 30]",0.901774,"[0.984482758621, 1.01551724138]"
6,0,19426,0,0,0,19850,0,0,0,"[[210.29876709, 516.383605957, -553.8125, -129...",3,False,../myNewFaxes/595146ed7f2583332b2f16e1,3302.514648,6,2,"[351, 48]",0.481037,"[0.963793103448, 0.953448275862]"
7,0,18499,0,0,0,0,0,0,0,"[[124.699768066, -84.7997817993, -763.84240722...",0,False,../myNewFaxes/593fedac91b5c942ab66606b,1426.728516,6,6,"[361, 41]",0.580246,"[0.984482758621, 1.00517241379]"
8,0,18715,0,0,0,19979,0,0,0,"[[8.77147102356, 193.254043579, 1423.42163086,...",2,False,../myNewFaxes/595693e54c97200c363a9670,3402.716309,6,3,"[378, 45]",0.340488,"[0.93275862069, 0.953448275862]"
9,0,16370,0,0,0,69525,0,0,0,"[[-999.438232422, -145.870773315, 1677.8071289...",2,False,../myNewFaxes/595fda5e4c97203191216f45,2365.101562,6,3,"[365, 27]",0.543648,"[0.922413793103, 1.01551724138]"


In [205]:
gtDF.head(20)
#gtDF.groupby('initial_refills').count() / float(gtDF.shape[0]-)
#gtDF.groupby('initial_refills').count() / float(gtDF.shape[0])
ar = np.array([218,243,107,395,32,40,11,1,9,3,1])
print ar / float(np.sum(ar))
gtDF.groupby('initial_refills').count()
# gtDF.groupby('initial_refills').count()


[ 0.20566038  0.22924528  0.1009434   0.37264151  0.03018868  0.03773585
  0.01037736  0.0009434   0.00849057  0.00283019  0.0009434 ]


,daw_code,days_supply,document_id,document_type,fax_id,prescription_id
initial_refills,,,,,,
0.0,565,582,582,582,582,582
1.0,239,243,243,243,243,243
2.0,107,107,107,107,107,107
3.0,391,395,395,395,395,395
4.0,31,32,32,32,32,32
5.0,39,40,40,40,40,40
6.0,11,11,11,11,11,11
10.0,1,1,1,1,1,1
11.0,9,9,9,9,9,9


In [239]:
gtDF.loc[gtDF['fax_id']=='595577cd4c972074d6676c24']
gtDF.loc[gtDF['fax_id']=='5911eae991b5c97f3ee908e0']
#we want to create an inner join on the tables. Need to first edit the imageName to just be the fax number
for i in range(resultsDF.shape[0]):
    resultsDF.set_value(i, 'imageName',resultsDF.iloc[i]['imageName'].replace("../myNewFaxes/", '').replace(".tiff", ''))
# resultsDF.head(4)

allInfo = gtDF.merge(resultsDF, right_on = 'imageName', left_on = 'fax_id')
print allInfo.shape
allInfo.head(10)
indices = allInfo['initial_refills'].notnull().index
#well index filtering certainly didn't work....
subtable = allInfo.iloc[indices][['initial_refills', 'predictedDigit', 'digitPictureNumber', 'numberOfPictures', 'imageName', 'Number of Additional Refills', 'templateMatchValue']]
correctRefills = 0
incorrectRefills = 0
correctFax = []
incorrectFax = []
zeroFax = []
actualNumber = []
predictedNumber = []
zeroWithMarking = 0
zeroNoMarking = 0
correctArray = [0,0,0,0,0,0,0,0,0,0]
incorrectActual = [0,0,0,0,0,0,0,0,0,0]
incorrectGuessed = [0,0,0,0,0,0,0,0,0,0]
indexErrors = []
for i in range(subtable.shape[0]):
    wasitzero = False
    #if not np.isnan(subtable.iloc[i]['initial_refills']) and not subtable.iloc[i]['initial_refills'] == 0 and subtable.iloc[i]['templateMatchValue'] < .9:
    actual = subtable.iloc[i]['initial_refills']
    if not np.isnan(subtable.iloc[i]['initial_refills']) and subtable.iloc[i]['templateMatchValue'] < .9:
        if subtable.iloc[i]['Number of Additional Refills'] < 5000:
            pass
#             predicted = 0
#             zeroNoMarking += 1
#             wasitzero = True
        else:
            predicted = subtable.iloc[i]['predictedDigit']
            if actual == 0:
                zeroWithMarking += 1
            actualNumber.append(int(actual))
            predictedNumber.append(predicted)
            if predicted == actual:
                correctRefills += 1
                #correctFax.append(subtable.iloc[i]['imageName'])
                correctFax.append(i)
                correctArray[int(actual)] += 1
            else:
                if wasitzero:
                    zeroFax.append((subtable.iloc[i]['imageName'], actual))   
                incorrectRefills += 1
                incorrectFax.append(subtable.iloc[i]['imageName'])
                try:
                    incorrectActual[int(actual)] += 1
                    incorrectGuessed[int(predicted)] += 1
                except IndexError:
                    indexErrors.append(actual)
print 'correct: ', correctRefills
print 'incorrect: ', incorrectRefills
print 'zero mark is ', zeroWithMarking
print 'zero no mark is ', zeroNoMarking
print subtable.loc[subtable['imageName'] == '590ce1c491b5c90b4c830ec9']
print zeroFax
newTab = subtable.loc[subtable.index.isin(correctFax)]
newTab.head(100)
print correctArray
print incorrectActual
print incorrectGuessed
# print correctFax
# print incorrectFax
#print actualNumber
#print predictedNumber
# plt.plot(actual,predicted)
# plt.xlabel("actual")
# plt.ylabel("predicted")
# plt.grid(True)
# plt.show()
#subtable.loc[subtable['imageName'] == '59089fd1acdd3f6fa04aa97e']
#45.5% success... to be fair that's better than 10% by chance so we're better than random happinstance! But a long way to go. We should find out how many of these actually said the refill though... 
#subtable.head(50)

(2035, 26)
correct:  414
incorrect:  542
zero mark is  188
zero no mark is  0
     initial_refills  predictedDigit  digitPictureNumber  numberOfPictures  \
621              1.0               5                   4                 5   

                    imageName  Number of Additional Refills  \
621  590ce1c491b5c90b4c830ec9                          3384   

     templateMatchValue  
621            0.294156  
[]
[27, 91, 61, 201, 5, 22, 7, 0, 0, 0]
[161, 125, 36, 172, 24, 9, 2, 0, 0, 0]
[21, 31, 119, 16, 5, 135, 141, 59, 2, 0]


In [158]:
#Found A Line
#table = gtDF.loc[gtDF['initial_refills'] > 9]
table = resultsDF.loc[resultsDF['foundALine'] == True]
print "total with a line: ", table.shape[0]
print "total number faxes: ", resultsDF.shape[0]
print "Percent with line: ", table.shape[0] / float(resultsDF.shape[0]) #percentage that had a line
table.head(30)

total with a line:  350
total number faxes:  2043
Percent with line:  0.17131669114


,30-Day Supply,90-Day Supply,Already Sent a Refill,Have Patient Contact My Office,No Longer a Patient,Number of Additional Refills,Other Reason for Denial,Patient Unknown to this Office,Refill Too Soon,digitConfidences,digitPictureNumber,foundALine,imageName,maxDigitConf,numberOfPictures,predictedDigit,templateLocation,templateMatchValue,templateRatio
1,2048,21949,0,0,0,23229,0,0,0,"[[648.535461426, 936.109313965, -343.333496094...",3,True,5931d010acdd3f4b517caa7e,4097.559570,6,3,"[366, 44]",0.596617,"[0.994827586207, 0.984482758621]"
5,22897,6511,0,256,0,37008,128,0,0,"[[-1043.94958496, 195.873092651, 1181.68444824...",2,True,593efd033839be1fa4d94d53,1934.181152,6,2,"[358, 30]",0.901774,"[0.984482758621, 1.01551724138]"
15,2023,10927,0,0,0,11032,202,0,1342,"[[537.171936035, 1050.73376465, -852.255065918...",0,True,594be2a791b5c90b96d690a3,1292.540894,6,6,"[388, 27]",0.628271,"[0.974137931034, 0.974137931034]"
22,0,10172,270,0,0,15354,0,0,276,"[[-1349.63928223, -545.208129883, 77.183486938...",1,True,590cf3240d16f82f00751228,2112.804688,6,3,"[352, 88]",0.526382,"[0.881034482759, 0.881034482759]"
27,0,11484,0,0,0,0,0,0,0,"[[418.745849609, 196.884643555, -499.04309082,...",0,True,5912204f3839be33075c540b,1438.099731,6,6,"[370, 38]",0.676312,"[0.963793103448, 0.963793103448]"
30,0,22066,0,0,0,0,3827,0,0,"[[454.500518799, 533.893920898, -532.750305176...",5,True,59417a81acdd3f11553dc5c4,1440.802002,6,2,"[354, 74]",0.520408,"[0.891379310345, 0.922413793103]"
33,0,13919,0,0,0,512,0,0,0,"[[2.80693244934, 389.547729492, -224.134231567...",5,True,595eb2e74c97203d2462e0b4,1030.375977,6,3,"[354, 29]",0.640285,"[0.943103448276, 1.01551724138]"
35,0,0,16308,0,144,0,0,0,0,"[[186.594924927, 683.516540527, -76.7035369873...",5,True,5963c55c68b2506888d7555a,1535.952637,6,1,"[372, 31]",0.496779,"[0.943103448276, 1.00517241379]"
36,0,8536,0,380,0,17462,0,108,1,"[[-220.405502319, 1864.04833984, -338.34045410...",5,True,59441debacdd3f4b5abf9824,3821.417236,6,2,"[372, 66]",0.664955,"[0.922413793103, 0.943103448276]"
37,13592,942,0,0,0,7308,1242,0,0,"[[-227.131271362, 414.00982666, 325.288299561,...",5,True,5939e9dd3839be7ac35254c2,1485.497314,6,2,"[362, 33]",0.662684,"[0.93275862069, 1.02586206897]"


In [190]:
resultsDF.loc[resultsDF['imageName'] == '59569d6a7f25837b0c46f9e6']

,30-Day Supply,90-Day Supply,Already Sent a Refill,Have Patient Contact My Office,No Longer a Patient,Number of Additional Refills,Other Reason for Denial,Patient Unknown to this Office,Refill Too Soon,digitConfidences,digitPictureNumber,foundALine,imageName,maxDigitConf,numberOfPictures,predictedDigit,templateLocation,templateMatchValue,templateRatio
221,0,5472,0,0,0,13743,0,0,0,"[[524.699401855, 17.407075882, -789.364074707,...",2,False,59569d6a7f25837b0c46f9e6,2311.594482,6,7,"[346, 40]",0.369299,"[0.963793103448, 0.974137931034]"


In [193]:
#30 Day Supply vs. 90 Day Supply
#Denied vs. Refill

resultsDF.head(4)
#don't care about pixels, copy paste was easiest
checkBoxPixels = {"30-Day Supply": [45,61,23,40], "90-Day Supply": [72,90,23,40], "Have Patient Contact My Office":[45,63,391,409], "No Longer a Patient": [73,91,391,409], "Already Sent a Refill": [101,119,391,409], "Patient Unknown to this Office": [129,146,391,409], "Refill Too Soon":[156,174,391,409], "Other Reason for Denial": [199,218,391,737]}
darkList = checkBoxPixels.keys()
#let's just do this dumbly, we don't even have that many to loop through
numberWithMoreThan1Box = 0
numberWithOneBox = 0
for index, row in resultsDF.iterrows():
    num = 0
    for title in darkList:
        if row[title] > 5000:
            
            num +=1 
    if num >1:
        numberWithMoreThan1Box += 1
    if num == 1:
        numberWithOneBox += 1
#print numberWithMoreThan1Box / float(resultsDF.shape[0])
#print numberWithOneBox / float(resultsDF.shape[0])

correct30 = 0
correct90 = 0
incorrect30 = 0
incorrect90 = 0
actually90 = 0
actually30 = 0
correctDenial = 0
incorrectDenial = 0
gtTotalDenied = 0
gtTotalRefilled = 0
denialBoxes = ["Have Patient Contact My Office", "No Longer a Patient", "Already Sent a Refill", "Patient Unknown to this Office", "Refill Too Soon", "Other Reason for Denial"]
sadness = []
i=0

# name = "5935dfdb91b5c97741037eea"
# # gtRow = gtDF.loc[gtDF['fax_id'] == name]
# gtRow = gtDF.loc[gtDF['fax_id'] == name].iloc[0]
# gtRow['daw_code']
#This is going to give an error, i think the key doesn't always exist. Investigate! 
#But we have to move on to the stupid google api now. Why am I incompetent....
# gtRow["initial_refills"]
for index, row in resultsDF.iterrows():
    name = row['imageName'].replace("../myNewFaxes/", '').replace(".tiff", '')
    try:
        takenCareOf = False
        gtRow = gtDF.loc[gtDF['fax_id'] == name].iloc[0]
        if row['templateMatchValue'] < .7: #make sure this is the right form, no funny business
            if gtRow['document_type'] == 'DeniedPrescription':
                gtTotalDenied += 1
            elif gtRow['document_type'] == 'Prescription':
                gtTotalRefilled += 1
            else:
                print "we don't know what happened: ", gtRow['document_type']
            #we want a list of which it checked off
            checkedOff = []
            for title in darkList:
                if row[title] > 5000:
                    checkedOff.append(title)
            #first let's check days supply
            if "30-Day Supply" in checkedOff and len(checkedOff) ==1:
                gtRow['document_type'] 
                #print gtRow['days_supply']
                if gtRow['document_type'] == 'Prescription' and gtRow['days_supply'] == 30:
                    correct30 += 1
                    takenCareOf = True
                else:
                    incorrect30 += 1
                    takenCareOf = True
            if "90-Day Supply" in checkedOff and len(checkedOff) ==1:
                gtRow['document_type'] 
                #print gtRow['days_supply']
                if gtRow['document_type'] == 'Prescription' and gtRow['days_supply'] == 90:
                    correct90 += 1
                    takenCareOf = True
                else:
                    incorrect90 += 1
                    takenCareOf = True
                    #print "incorrect90 is ", name
            ####denial check
            
            if not "30-Day Supply" in checkedOff and not "90-Day Supply" in checkedOff and not row["Number of Additional Refills"] > 3000:
                if gtRow['document_type'] == 'DeniedPrescription':
                    correctDenial += 1
                    takenCareOf = True
                else:
                    incorrectDenial += 1
                    takenCareOf = True
                    #print "incorrect Denial on ", name
#             for reason in denialBoxes:
#                 if reason in checkedOff:
#                     if not "30-Day Supply" in checkedOff and not "90-Day Supply" in checkedOff:
#                         #it thinks it's denied
#                         if gtRow['document_type'] == 'DeniedPrescription':
#                             correctDenial += 1
#                             TakenCareOf = True
#                         else:
#                             incorrectDenial += 1
#                             TakenCareOf = True
#                             print "incorrect Denial on ", name
#                         break
            if not takenCareOf:
                sadness.append(index)
    except IndexError:
        print "INDEX ERROR ON ", name

print "correct30: ",correct30
print "correct90: ",correct90
print "incorrect 30: ",incorrect30
print "incorrect 90: ",incorrect90
print "correctDenial: ", correctDenial
print "incorrectDenial: ",incorrectDenial
print "gtTotalDenied: ", gtTotalDenied
print "gtTotalRefilled: ", gtTotalRefilled
totalGuessed = correct30+correct90+correctDenial+incorrect30+incorrect90+incorrectDenial
totalCorrect = correct30+correct90+correctDenial
print "totalGuessed: ", totalGuessed
print "totalCorrect: ", totalCorrect
print "percentCorrect: ", float(totalCorrect)/totalGuessed
print "percentGuessed: ", float(totalGuessed)/resultsDF.shape[0]
#print sadness
resultsDF.iloc[sadness]
    

INDEX ERROR ON  5949835b0d16f8578fbdc145
INDEX ERROR ON  5965661b7f25832ca692172e
INDEX ERROR ON  591b0cff0d16f83daf6a95ea
INDEX ERROR ON  59385a190d16f8605df07aaf
INDEX ERROR ON  593aa66e0d16f8083d795bb9
we don't know what happened:  DemographicInfo
INDEX ERROR ON  59402fdaacdd3f778fe41c4f
INDEX ERROR ON  5955823368b25020bc450b60
INDEX ERROR ON  5953ed104c97205bb78a231c
correct30:  139
correct90:  819
incorrect 30:  16
incorrect 90:  76
correctDenial:  456
incorrectDenial:  27
gtTotalDenied:  490
gtTotalRefilled:  1140
totalGuessed:  1533
totalCorrect:  1414
percentCorrect:  0.922374429224
percentGuessed:  0.750367107195


,30-Day Supply,90-Day Supply,Already Sent a Refill,Have Patient Contact My Office,No Longer a Patient,Number of Additional Refills,Other Reason for Denial,Patient Unknown to this Office,Refill Too Soon,digitConfidences,digitPictureNumber,foundALine,imageName,maxDigitConf,numberOfPictures,predictedDigit,templateLocation,templateMatchValue,templateRatio
18,0,10958,0,0,90,22509,11710,0,358,"[[-110.825149536, 424.457763672, -534.79315185...",2,False,59418abd91b5c93d418c5ade,1706.949097,6,0,"[368, 33]",0.664929,"[0.943103448276, 1.01551724138]"
128,0,3575,0,0,0,27427,255,0,0,"[[-243.230102539, 624.981323242, 79.1346588135...",3,False,590b4b850d16f823de417c1b,3504.783203,6,3,"[350, 48]",0.477409,"[0.953448275862, 0.953448275862]"
136,0,796,0,47,5,3322,0,0,158,"[[800.351257324, 931.852539062, -790.282043457...",0,False,594ad2540d16f83d20e8c376,1599.678467,6,6,"[367, 60]",0.479110,"[0.953448275862, 0.943103448276]"
151,0,9739,0,0,0,0,6182,0,0,"[[283.425476074, 434.719085693, -1276.89660645...",0,False,594a9d80acdd3f2963170dc8,1795.492676,6,6,"[343, 91]",0.459858,"[0.912068965517, 0.881034482759]"
160,0,10337,0,0,0,0,6648,0,2112,"[[409.70199585, 578.034240723, -857.305480957,...",0,True,592d9baf0d16f8340fbf7945,1627.131226,6,6,"[344, 70]",0.662912,"[0.912068965517, 0.901724137931]"
163,23365,0,0,116,0,26320,9580,0,97,"[[167.587020874, 1550.36010742, 301.663513184,...",2,False,5936c96facdd3f601ae87c10,5700.825195,6,3,"[353, 60]",0.686075,"[0.974137931034, 0.943103448276]"
175,654,12546,4349,7704,7116,13764,9962,4080,5817,"[[-67.7652130127, 636.91204834, 2.12344741821,...",4,True,59079f8a0d16f86771961686,1975.608887,5,3,"[373, 61]",0.378076,"[0.943103448276, 0.891379310345]"
184,0,8805,0,0,0,22322,15757,0,0,"[[1260.83044434, 690.203674316, -1056.78198242...",0,False,595ed25568b2502bcb77102e,2358.920166,6,6,"[378, 51]",0.511745,"[0.953448275862, 0.943103448276]"
236,0,12312,0,0,0,0,7899,0,9,"[[397.138122559, 803.536682129, -115.567001343...",4,False,5930646d91b5c96f56cc17ff,2850.844238,5,1,"[371, 75]",0.554225,"[0.93275862069, 0.922413793103]"
260,0,310,0,22914,0,5786,0,0,0,"[[-243.608444214, 176.70640564, 698.982299805,...",5,False,5931fa59acdd3f46210157e2,1405.401001,6,5,"[354, 47]",0.685792,"[0.974137931034, 0.994827586207]"


In [206]:
 #####Testing new mnist data. 
####one-time load
with open('../numWithBoxesData.txt') as data_file:
    data = json.load(data_file)

In [216]:
#now we will extract the beautiful matrices. 
import tensorflow as tf
import cv2

W_conv1 = tf.reshape(data["W_conv1"], [8,8,1,32])
b_conv1 = tf.reshape(data["b_conv1"], [32])
W_conv2 = tf.reshape(data["W_conv2"], [5,5,32,64])
b_conv2 = tf.reshape(data["b_conv2"], [64])
W_conv3 = tf.reshape(data["W_conv3"], [3,3,64,64])
b_conv3 = tf.reshape(data["b_conv3"], [64])
W_fc1 = tf.reshape(data["W_fc1"], [5*8*64, 2048])
b_fc1 = tf.reshape(data["b_fc1"], [2048])
W_fc2 = tf.reshape(data["W_fc2"], [2048,13])
b_fc2 = tf.reshape(data["b_fc2"], [13])

#now set up the model/multiplication stuff
x = tf.placeholder(tf.float32, [None,36,64])
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME') #ima guess W is our feature vector. Always make first and last 1 because why would we skip an image (batch) or... idk what the last is honestly but don't want to skip any

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides = [1, 2, 2, 1], padding = 'SAME')

x_image = tf.reshape(x, [-1, 36, 64, 1])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)
h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3)
h_pool3 = max_pool_2x2(h_conv3)
h_pool3_flat = tf.reshape(h_pool3, [-1, 5*8*64])

h_fc1 = tf.nn.relu(tf.matmul(h_pool3_flat, W_fc1) + b_fc1)
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2 #these are outputs not normalized yet

y_probabilities = tf.nn.softmax(y_conv) #I think this should make it probabilities, but it only makes sense if we feed in one image


###UGHHHH ok we need to preprocess these again
####So that we remove lines and rotate them correclty
#GRRRRRR
#####Alright what we want is to probably just get a list of image names that have a number
#then output to json file. Then read that to our other program rewriting a new, temporary read digit function
#and then we shall know.  SIGHH this is grunt work and it makes me sad 
###now read the number for one image example
checkBoxPixels = {"30-Day Supply": [45,61,23,40], "90-Day Supply": [72,90,23,40], "Number of Additional Refills": [100,118,23,66], "Have Patient Contact My Office":[45,63,391,409], "No Longer a Patient": [73,91,391,409], "Already Sent a Refill": [101,119,391,409], "Patient Unknown to this Office": [129,146,391,409], "Refill Too Soon":[156,174,391,409], "Other Reason for Denial": [199,218,391,737], "DigitPrediction":[5,131,14,78]}
def calculatePixels(key, myRatio, minLoc): #minLoc is coordinate where the template should sit, myRatio is the scaling of the template, key is the box we're looking at
    pixels = checkBoxPixels[key]
    #print "pixels are ", pixels
    #print "myRatio is ", myRatio
    pixels = np.array([pixels[0] * myRatio[0], pixels[1] * myRatio[0], pixels[2] * myRatio[1], pixels[3] * myRatio[1]]).astype(int) #this should give us the coordinates relative to top corner of resized template
    pixels = np.array([pixels[0] + minLoc[0], pixels[1] + minLoc[0], pixels[2] + minLoc[1], pixels[3] + minLoc[1]])
    print pixels
    #in the order xtop xbottom yleft yright
    return pixels
def readNumber(myRatio, minLoc, mainImage): 
    myPixels = calculatePixels("DigitPrediction", myRatio, minLoc)
    boxImage = mainImage[myPixels[0]:myPixels[1], myPixels[2]:myPixels[3]]
    boxImage = cv2.resize(boxImage, dsize=(36,64))
    print boxImage.shape
    with tf.Session() as sess:
        yProb = sess.run(y_probabilities, feed_dict = {x: boxImage, keep_prob: 1.0}) #run on the slidy thing and on the center of mass thing. We'll end up taking the thing it's most confident of anyway
    return yProb



In [227]:
allInfo = gtDF.merge(resultsDF, right_on = 'imageName', left_on = 'fax_id')
allInfo.head(5)

# with open('turningDFToJson.txt', 'w') as myFile:
#     json.dump(allInfo.to_json(), myFile)


# for i in range(allInfo.shape[0]):
#     imageName = '../myNewFaxes/'+str(allInfo.iloc[i]['fax_id'])
#     if allInfo.iloc[i]['Number of Additional Refills'] > 5000:
#         image = cv2.imread(imageName)
#         print readNumber(allInfo.iloc[i]['templateRatio'], allInfo.iloc[i]['templateLocation'], image)
#     else:
#         print "skipping this one"

,daw_code,days_supply,document_id,document_type,fax_id,initial_refills,prescription_id,30-Day Supply,90-Day Supply,Already Sent a Refill,...,digitConfidences,digitPictureNumber,foundALine,imageName,maxDigitConf,numberOfPictures,predictedDigit,templateLocation,templateMatchValue,templateRatio
0,0.0,0.0,591071991e414f6d5024e3ab,Prescription,591071983839be395b31b008,0.0,5911b18c1e414f1034de24af,9110,1998,0,...,"[[-151.108016968, -268.71496582, -818.60546875...",2,False,591071983839be395b31b008,2514.433838,6,0,"[363, 64]",0.664787,"[0.953448275862, 0.943103448276]"
1,0.0,90.0,590744071e414f2968944476,Prescription,590744073839be687f1d5ace,1.0,5907473191b5c96c3bbb33ee,2852,17209,0,...,"[[662.586853027, 766.337097168, -802.482727051...",0,False,590744073839be687f1d5ace,2121.521973,6,6,"[346, 77]",0.443199,"[0.912068965517, 0.901724137931]"
2,0.0,90.0,590744d71d0f173c99d0f611,Prescription,590744d73839be57c71d5b8e,3.0,5909e7fd1e414f24689409a7,510,893,0,...,"[[-59.8352622986, 250.931015015, -281.70867919...",4,False,590744d73839be57c71d5b8e,1585.285522,6,7,"[0, 0]",1.000000,"[0.85, 0.85]"
3,0.0,0.0,590745bb1e414f29689444a0,Prescription,590745bb91b5c95d34bb3409,0.0,590773a31e414f5cb53eeb59,0,15122,0,...,"[[561.541320801, 461.617645264, -339.859558105...",5,False,590745bb91b5c95d34bb3409,2189.101318,6,2,"[359, 87]",0.387908,"[0.870689655172, 0.881034482759]"
4,NaN,NaN,590749d81e414f6e715c937d,DeniedPrescription,590749d83839be0ee21d5b53,NaN,NaN,0,6456,0,...,"[[-59.8352622986, 250.931015015, -281.70867919...",5,True,590749d83839be0ee21d5b53,1267.106201,6,7,"[0, 0]",1.000000,"[0.85, 0.85]"


In [230]:
# with open('turningDFToJson.txt') as data_file:
#     allInfo = json.load(data_file)
# theTableIWant = pd.read_json(allInfo)
# theTableIWant.head(5)
# theTableIWant
row  = allInfo.loc[allInfo['fax_id']=='591071983839be395b31b008']
row['Number of Additional Refills'].iloc[0]

25449